<div style="
    background-color: #f7f7f7;
    background-image: url(''), url('') ;
    background-position: left bottom, right top;
    background-repeat: no-repeat,  no-repeat;
    background-size: auto 60px, auto 160px;
    border-radius: 5px;
    box-shadow: 0px 3px 1px -2px rgba(0, 0, 0, 0.2), 0px 2px 2px 0px rgba(0, 0, 0, 0.14), 0px 1px 5px 0px rgba(0,0,0,.12);">

<h1 style="
    color: #2a4cdf;
    font-style: normal;
    font-size: 2.25rem;
    line-height: 1.4em;
    font-weight: 600;
    padding: 30px 200px 0px 30px;">
    Querying the NOMAD Archive and performing artificial-intelligence modeling</h1>

<p style="font-size: 1.25em; font-style: italic; padding: 5px 200px 30px 30px;">
    Luigi Sbailò, Matthias Scheffler, Luca M. Ghiringhelli</p>
</div>

<div style="margin: 10px;">
    <img style="float: left; margin: 5px 20px 0px 0px;" src="assets/logos/hu-berlin.svg" width="110">
    <img style="float: left; margin: 0px 20px 0px 0px;" src="assets/logos/nomad.svg" width="110">
    <img style="float: left; margin: 15px 20px 0px 0px;" src="assets/logos/nomad-infrastructure.svg" width="120">
    <img style="float: left; margin: 5px 20px 0px 10px;" src="assets/logos/mpcdf.svg" width="270">
</div>
<p style="text-align: right; padding: 0px 10px 10px 0px;">[Last updated: September 19, 2023]</p>

In this tutorial, we show how to query the [NOMAD Archive](https://nomad-lab.eu/prod/v1/gui/search/entries) and perform data analysis on the retrieved data.


## Preliminary operations

We load the following packages that are all available in the virtual environment containing the Jupyter notebooks in the NOMAD AI toolkit. Among the loaded packages, we highlight `sklearn`, i.e., scikit-learn, one of the most popular machine-learning packages, and `pandas`, a popular tool for data handling and analysis.


In [ ]:
import json
import shutil
import itertools
import pathlib

import numpy as np
import pandas as pd

from sklearn import preprocessing

import matplotlib.pyplot as plt
import plotly.express as px

from ase.data import chemical_symbols
from atomic_features_package.atomic_properties import periodictable
from atomic_features_package.atomic_properties import atomic_properties_dft as ap
from atomic_features_package.atomic_properties import atomic_properties_pymat as pymat

from query_nomad_archive.visualizer import Visualizer

In [ ]:
import nest_asyncio

from nomad import config
from nomad.datamodel import EntryArchive
from nomad.client.archive import ArchiveQuery

nest_asyncio.apply()
config.client.url = "http://nomad-lab.eu/prod/v1/staging/api"

You can use already download data from local cache. Downloading the data from the archive takes about 10 mins.

In [ ]:
use_cache = True

We maintain a `nomad` package that can be imported in all notebooks of the AI Toolkit.


The `nomad` package allows to retrieve data from the NOMAD Archive by means of a script, as shown below. In this script, we insert metadata characterizing the materials that we aim to retrieve. In this case, we select ternary materials containing oxygen. We also request that simulations were carried out using the VASP code using GGA exchange-correlation (xc) functionals. Values are retrieved from the simulation run that found geometrically convergence wihin a desird threshold value of.


The modules `atomic_properties_dft` and `atomic_properties_pymat` make available atomic properites of the elements of the periodic table, to be used as features in data analytics. The sources of these atomic proporties are DFT calculations performed by the NOMAD team and <a href="https://pymatgen.org/" target="_blank">pymatgen</a>, respectively.


We have developed a visualization tool that allows us to visualize atomic properites of all elements accross periodic table as a heatmap. Currently, this tool is able to visualize atomic properties acessible from `atomic_properties_dft` module. Below is an example for the data calculated via the HSE06 functional and spinless settings.

This module can be used as follows. From the dropdown menu, one can select which property one is interested to check and the table is updated automatically to show the corresponding heatmap.


In [ ]:
periodictable.heatmap(Spin="False", method="pbe")

## Querying the NOMAD Archive


In [ ]:
max_entries = 8862

query = {
    "results.method.simulation.program_name": "VASP",
    "results.material.elements": ["O"],
    "results.material.symmetry.crystal_system:any": ["cubic"],
    "results.material.n_elements": {"lte": 3, "gte": 3},
    "results.method.simulation.dft.xc_functional_type:any": ["GGA"],
    "results.properties.geometry_optimization": {
        "final_energy_difference": {"lte": 2e-22, "gte": 1e-30}
    },
}
required = {
    "results": {"material": {"symmetry": {"space_group_number": "*"}}},
    "workflow": {
        "geometry_optimization": {
            "final_energy_difference": "*",
        },
        "calculation_result_ref": {
            "system_ref": {
                "chemical_composition_reduced": "*",
                "atoms": {
                    "species": "*",
                    "lattice_vectors": "*",
                    "positions": "*",
                },
            },
        },
    },
}

query = ArchiveQuery(
    query=query, required=required, page_size=100, results_max=max_entries
)

We have defined the variable `query`, which allows to perform our query.
All quantities defined in the `required` field are fetched during the query. For example, we can see quantities as `chemical_composition_reduced` which gives the composition of the material or `atoms.positions` that contains the positon of all atoms after geometric convergence.
We notice that the variable `query` contains a number of other variables: the `max_entries` value sets the maximum number of entries that can be retrieved; the `per_page` value indicates the number of entries fetched at each API call.


In this tutorial, we use machine learning tools to investigate properties of materials. In particular, we aim to predict the atomic density of the materials as a function of some primary features such as the atomic number.

To retrieve data and place it within a framework, we use a 'for' loop that iteratively fetches all entries up to the maximum value, which is given by 'max_entries'. In addition, we also make sure the entry contains the simulation cell value which we are interested in, and that all elements in the material have an admissible atomic number.


In the next cell, we download data from the NOMAD Archive. **As we query a large number of elements, this operation can be time consuming. Hence, we have cached the results of the following query, and data can be loaded with a command given in the subsequent cell.**


In [ ]:
if not use_cache:
    query.fetch()
    results = []
    while True:
        result = query.download(100)
        if len(result) == 0:
            break
        results.extend(result)

    # (optional) Save the results to a file
    # filename = 'data/query_nomad_archive/results.json'
    # with open(filename, 'w') as f:
    #     json.dump({'data': [entry.m_to_dict() for entry in results]}, f)

else:
    with open("data/query_nomad_archive/results.json") as f:
        data = json.load(f)

    results = [EntryArchive.m_from_dict(entry) for entry in data["data"]]

len(results)

Filtering out some of the non relevant calculations:

In [ ]:
def valid_calculation(entry):

    system_ref = entry.workflow[0].calculation_result_ref.system_ref
    elements = system_ref.atoms.species

    if min(elements) < 1 or max(elements) > 118:
        return False

    return True

results = [entry for entry in results if valid_calculation(entry)]
len(results)

We instantiate the `atomic_properties_dft` (imported as `ap` here) classes with the atomic symbol by first specifying DFT spin setting employed and fucntional for evaluation of atomic properties.
Functional for which this properties accessible are 'hse06', 'pbe','pbe0','pbesol','pw-lda','revpbe'.
The spin setting could be either 'True' or 'False'.


In [ ]:
ap.method(method="pbe", Spin="False")

In [ ]:
def get_features(ind, entry):
    features = dict()

    system_ref = entry.workflow[0].calculation_result_ref.system_ref
    elements = system_ref.atoms.species

    # Dimensions of the cell are rescaled to angstroms.
    lattice_vectors = system_ref.atoms.lattice_vectors.m_as("angstrom")

    features["Formula"] = system_ref.chemical_composition_reduced
    features["Convergence_energy_diff"] = entry.workflow[0].geometry_optimization.final_energy_difference.m
    features["Space_group_number"] = int(entry.results.material.symmetry.space_group_number)

    # The total number in the array 'elements' gives the total number of atoms.
    n_atoms = len(elements)

    # The volume of the cell is obtained as scalar triple product of the three base vectors.
    # The triple scalar product is obtained as determinant of the matrix composed with the three vectors.
    cell_volume = np.linalg.det(lattice_vectors)

    # The atomic density is given by the number of atoms in a unit cell.
    features["Atomic_density"] = n_atoms / cell_volume

    # The ternary materials are composed by Oxygen and two other elements labeled as A,B.
    # Variables 'Z_A','Z_B' contain the atomic number of the elements A,B.
    Z_A, Z_B = np.unique(elements[elements != 8])

    if Z_A > Z_B:
        Z_A, Z_B = Z_B, Z_A

    features["Atomic_number_A"] = Z_A
    features["Atomic_number_B"] = Z_B

    # The fraction of atoms of a specific element within the material, that is also given by the stochiometric ratio.
    features["Fraction_O"] = np.count_nonzero(elements == 8) / n_atoms
    features["Fraction_A"] = np.count_nonzero(elements == Z_A) / n_atoms
    features["Fraction_B"] = np.count_nonzero(elements == Z_B) / n_atoms

    # We instantiate the atomic_properties_dft(imported as ap here) classes with the atomic symbol.
    # Similarly we instantiate the atomic_properties_pymat(imported as pymat here) classes with the atomic symbol
    # These classes allow to retrieve atoms properties, in this example we fetch the element ionization energy,atomic radius
    # from atomic_properties_dft module and element name,element weight  from atomic_properties_pymat module

    label_A = chemical_symbols[Z_A]
    label_B = chemical_symbols[Z_B]

    pymat_A = pymat.symbol(label_A)
    pymat_B = pymat.symbol(label_B)

    features["Element_A_name"] = pymat_A.atomic_element_name
    features["Element_B_name"] = pymat_B.atomic_element_name

    features["Weight_A"] = pymat_A.atomic_mass
    features["Weight_B"] = pymat_B.atomic_mass

    A = ap.symbol(label_A)
    B = ap.symbol(label_B)

    # distance values are in angstroms so we convert these to pm in this eg
    features["Atomic_radius_A"] = A.atomic_r_val[0].m_as("pm")
    features["Atomic_radius_B"] = B.atomic_r_val[0].m_as("pm")

    # energy value obtained is in joules so we convert to eV
    features["Ionenergy_A"] = A.atomic_ip.m_as("eV")
    features["Ionenergy_B"] = B.atomic_ip.m_as("eV")

    # energy value obtained is in joules so we convert to eV
    features["El_affinity_A"] = A.atomic_ea.m_as("eV")
    features["El_affinity_B"] = B.atomic_ea.m_as("eV")

    # energy value obtained is in joules so we convert to eV
    features["Homo_A"] = A.atomic_hfomo.m_as("eV")
    features["Homo_B"] = B.atomic_hfomo.m_as("eV")
    features["Lumo_A"] = A.atomic_hfomo.m_as("eV")
    features["Lumo_B"] = B.atomic_hfomo.m_as("eV")

    return features


# At each iteration, we add to the dataframe one row that contains the A,B elements in the material and a number of other material properties.
features = [get_features(ind, entry) for ind, entry in enumerate(results)]

len(features)

We define a 'Pandas' dataframe that contains all fetched data.


In [ ]:
df = pd.DataFrame.from_records(features)

Pandas dataframes include the 'describe' method which gives an overview about the composition of the dataset.


In [ ]:
df.describe()

We are particularly interested in materials properties and how they can be inferred from the solely atomic composition of a specific material. In our query, we have retrieved two materials properties, namely the _'Atomic_density'_ and the _'Space_group_number'_. Before performing any machine learning analysis, it is interesting to visualize the distribution of these values using a histogram. Pandas allows for a straighforward visualization of histograms constructed with columns dataframe values.


In [ ]:
df.hist(column="Space_group_number", bins=range(200, 231), align="left", figsize=(10, 4))
plt.xticks(range(200, 231), rotation=70)
plt.xlim([200, 230])
plt.show()

We notice above that retrieved materials can be mainly classified into two distinct space group numbers, i.e. the sapce group number 221 and 227. It would be interesting to see whether such distinction involves that materials belonging in the same space group share similar atomistic properties, while the ones belonging in different space groups are distinct also on an atomistic level. This is the scope of clustering, and it will be the object of an in-depth analysis below.

Now, we keep inspecting the dataframe values and plot a histogram containing the values of the atomic density.


In [ ]:
df.hist(column="Atomic_density", bins=50, figsize=(10, 4))
plt.show()

The histogram above shows that the atomic density is mainly distributed around a value of 0.07 Angstroms$^{-1}$. Such distribution might seem the result of a random extraction, but we aim to find an AI model that is able to make high-resolution predictions for the atomic density based only on the atomic composition of the material.


In order to build an AI model that makes reliable predictions, we should make sure that each entry has a different representation. In this case, as we are interested in predicting material properties from the atomic composition, the chemical composition of the material is an ideal representation for the dataframe entries. However, we might have different entries with the same chemical composition, because e.g. simulations were performed for the same material with different settings that were not included among the filters of our query. Each of these simulations might have produced a slightly different value of the resulting atomic density of the material. As data is taken from heterogeneous simulations which were carried out in different laboratories, we do not aim to evaluate all possible parameters for each simulation. Hence, we average the atomic density value over all materials with the same chemical composition.

After averaging (or _grouping_) data is placed in a different dataframe '_df_grouped_', where each entry represents a different compound.


In [ ]:
df_grouped = df.groupby(
    ["Formula", "Element_A_name", "Element_B_name", "Space_group_number"]
).mean()

df_grouped = df_grouped.reset_index(
    level=["Element_A_name", "Element_B_name", "Space_group_number"]
)

df_grouped["Replicas"] = (
    df.groupby(["Formula", "Element_A_name", "Element_B_name", "Space_group_number"])
    .count()["Atomic_density"]
    .values
)

With data placed in a dataframe, we can carry out our machine learning analysis.


# Example of unsupervised machine learning: Clustering and dimension reduction

---


Firstly, we perform an explorative analysis to understand how data is composed and organized. Hence, we use
unsupervised learning to extract from the dataset clusters of materials with similar properties.

We define the list of features that are used for clustering including only the stochiometric ratio and the atomic number. Our aim is to use unsupervised learning for understanding whether the defined descriptors are sufficient for structuring the dataset.


In [ ]:
clustering_features = []
clustering_features.append("Fraction_A")
clustering_features.append("Fraction_B")
clustering_features.append("Fraction_O")
clustering_features.append("Atomic_number_A")
clustering_features.append("Atomic_number_B")
clustering_features.append("El_affinity_A")
clustering_features.append("El_affinity_B")
# clustering_features.append('Ionenergy_A')
# clustering_features.append('Ionenergy_B')
# clustering_features.append('Homo_A')
# clustering_features.append('Homo_B')
# clustering_features.append('Lumo_A')
# clustering_features.append('Lumo_B')
# clustering_features.append('Weight_A')
# clustering_features.append('Weight_B')
# clustering_features.append('Atomic_radius_A')
# clustering_features.append('Atomic_radius_B')

df_clustering = preprocessing.scale(df_grouped[clustering_features])

As clustering algorithm we use HDBSCAN, that is described in:

<div style="padding: 1ex; margin-top: 1ex; margin-bottom: 1ex; border-style: dotted; border-width: 1pt; border-color: blue; border-radius: 3px;">
R.J.G.B. Campello, D. Moulavi, J. Sander: <span style="font-style: italic;">Density-Based Clustering Based on Hierarchical Density Estimates</span>,  Springer Berlin Heidelberg, (2013).</div>
The only input parameter that this algorithm requires is the minimum size of each cluster.

To achieve a more accurate cluster definition, HDBSCAN is able to detect all those points that are hardly classified into a specific cluster, which are labeled as 'outliers'.

The implementation of the algorithm that we use is taken from https://pypi.org/project/hdbscan/.


In [ ]:
import hdbscan

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=60, min_samples=50)
clusterer.fit(df_clustering)

print(f"The algorithm finds {str(clusterer.labels_.max() + 1)} clusters.")

In [ ]:
cluster_labels = clusterer.labels_
df_grouped["Cluster_label"] = cluster_labels

To visualize our multidimensional data, we need to project it onto a two-dimensional manifold.
Hence, we use the TSNE embedding algorithm.


In [ ]:
from sklearn.manifold import TSNE

embedding = TSNE(n_components=2).fit_transform(df_clustering)

df_grouped["x_emb"] = embedding[:, 0]
df_grouped["y_emb"] = embedding[:, 1]

We maintain the Visualizer, a dedicated tool for an interactive visualization of the data retrieved from the Archive.

The Visualizer shows compounds belonging in different clusters with different colors.
Outliers by default are not shown on the map, but outliers visualization can be activated by clicking the 'Outliers' label on the right of the map.
The color of the markers can also represent a specific target property of the material that can be selected from the 'Marker colors' dropdown menu.
Target properties which can be displayed with different colors are inserted as a list of 'color_features'.
After selecting a specific property, a new menu will appear to choose the color scale to be used for visualizing the different values of that target property.

To prevent data overloading, only a fraction of the whole dataset is initially visualized.
This parameter can be adjusted modifing the fraction value on top of the map, up to select all entries in the dataset.

By hovering over the map, the Visualizer shows which compound corresponds to each point, and its number of 'Replicas'.  
Replicas represents the number of entries from the original dataset (before grouping) which have the same chemical composition.
It is also possible to select an additional number of features in the 'hover_features' list which are displayed while hovering.

Clicking on any of the points in the map automatically shows the 3D chemical structure of the material in one of the windows below.
Note that at each time the 'Display' button is clicked, a different structure with the same chemical composition is visualized.
A new structure is shown up to the 'Replicas' number.
This allows to inspect all possible structures in the dataset.

Furthermore, the chemical formula of a specific compound can be manually written in the 'Compound' textbox, and clicking the 'Display' button will both show the 3D chemical structure and mark the exact position of the compound on the map with a cross.
The Compound textbox includes autocompletion, which allows to inspect all materials in the dataset inserting partial formulae.

Lastly, the Visualizer offers a number of utils for producing high-quality plots of the map, which are displayed after clicking the button just below the map.


In [ ]:
from query_nomad_archive.visualizer import Visualizer

In [ ]:
hover_features = []
hover_features.append("Atomic_number_A")
hover_features.append("Atomic_number_B")
hover_features.append("Space_group_number")
hover_features.append("Atomic_density")
hover_features.append("Replicas")
hover_features.append("Cluster_label")

color_features = []
color_features.append("Atomic_density")
color_features.append("Space_group_number")

Visualizer(df, df_grouped, hover_features, color_features).view()

Using the visualizer we can analyse the composition of the different clusters extracted.
We have selected the atomic density and the space group number as color features.
This allows to inspect how these values vary within clusters.
The values of these features show ordered structures within clusters, that is particularly interesting because these features were not used by the clustering algorithm.
This suggests that the atomic features used for clustering are sufficient to describe certain properties of the whole material such as the most stable structure or the atomic density.
Therefore, we might imagine that there is a functional form capable to infer the defined materials properties from our atomic features, and we can train a supervised machine learning model to find such relationship.

The space group number of the elements in each cluster is shown below firstly as a list of values, then using a pie chart.
We can clearly notice that in each cluster there is one space group number that is predominant respect to all the others.
This means that we can label the different clusters with a characteristic space group number.


In [ ]:
df_count_groups = (
    df_grouped.loc[df_grouped["Cluster_label"] != -1]
    .groupby(["Cluster_label", "Space_group_number"])
    .describe()["Atomic_density"][["count"]]
)

df_count_groups = df_count_groups.reset_index()

n_clusters = df_grouped["Cluster_label"].max() + 1

df_count_groups

In [ ]:
for cl in range(n_clusters):
    df_cluster = df_count_groups.loc[df_count_groups["Cluster_label"] == cl]
    fig = px.pie(df_cluster, values="count", names="Space_group_number")
    fig.show()

The pie chart below includes the space group number of all elements classified as outliers. We can clearly see that the outliers include all different space group numbers, and it is not possible to identify a predominant space group number characteristic of the group of outliers.


In [ ]:
df_cluster = (
    df_grouped.loc[df_grouped["Cluster_label"] == -1]
    .groupby(["Space_group_number"])
    .describe()["Atomic_density"][["count"]]
    .reset_index()
)

px.pie(df_cluster, values="count", names="Space_group_number")

# Example of supervised machine learning: Random forest

---


Finally, we aim to use the large data set of materials that we have retrieved from the NOMAD Archive to train an AI model.
Previous findings obtained with the unsupervised analysis suggest that it is possible to train a model to predict materials properties using only atomic features.
The trained model can then be used to predict properties of yet unknown materials from the knowledge of its constituent atoms.
In this specific case, we aim to predict the average atomic density.

We use the Random forest method. Random forest is available from the scikit-learn package.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

We select all atomic properties as primary features and the atomic density of the material as target feature.


In [ ]:
ML_primary_features = []
ML_primary_features.append("Fraction_A")
ML_primary_features.append("Fraction_B")
ML_primary_features.append("Fraction_O")
ML_primary_features.append("Atomic_number_A")
ML_primary_features.append("Atomic_number_B")
ML_primary_features.append("El_affinity_A")
ML_primary_features.append("El_affinity_B")
ML_primary_features.append("Ionenergy_A")
ML_primary_features.append("Ionenergy_B")
ML_primary_features.append("Atomic_radius_A")
ML_primary_features.append("Atomic_radius_B")
# ML_primary_features.append('Homo_A')
# ML_primary_features.append('Homo_B')
# ML_primary_features.append('Lumo_A')
# ML_primary_features.append('Lumo_B')
# ML_primary_features.append('Weight_A')
# ML_primary_features.append('Weight_B')


ML_target_features = []
ML_target_features.append("Atomic_density")

Our dataset is divided into a train set and a test set. The model is trained only with the train set, while ignoring the values in the test set. This allows to test the prediction capability of the model on data that have not been seen.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df[ML_primary_features], df[ML_target_features], test_size=0.2
)

We train here the model.


In [ ]:
random_regressor = RandomForestRegressor(
    n_estimators=100, 
    max_depth=100, 
    max_features=5, 
    min_samples_split=5, 
    random_state=0
)
random_regressor.fit(X_train, y_train.to_numpy().ravel())

After training, we check the accuracy of the model.


In [ ]:
y_predict = random_regressor.predict(X_test)
print(
    f"The Ai model predicts the atomic density on a test set with an average error of ",
    f"{np.mean(np.abs(y_predict - y_test.to_numpy().flatten())):.4f} Angstroms^-1."
)

In [ ]:
y_predict = random_regressor.predict(X_train)
print(
    "The Ai model predicts the atomic density on a training set with an average error of ",
    f"{np.mean(np.abs(y_predict - y_train.to_numpy().flatten())):.4f} Angstroms^-1."
)

In [ ]:
y_predict = random_regressor.predict(X_test)
X_test = X_test.assign(Atomic_density=y_predict)

df_A_pred = X_test[["Atomic_number_A", "Atomic_density"]].rename(
    columns={"Atomic_number_A": "Atomic_number"}
)

df_B_pred = X_test[["Atomic_number_B", "Atomic_density"]].rename(
    columns={"Atomic_number_B": "Atomic_number"}
)

df_AB_pred = pd.concat([df_A_pred, df_B_pred], ignore_index=True)

df_A = df[["Atomic_number_A", "Atomic_density"]].rename(
    columns={"Atomic_number_A": "Atomic_number"}
)

df_B = df[["Atomic_number_B", "Atomic_density"]].rename(
    columns={"Atomic_number_B": "Atomic_number"}
)

df_AB = pd.concat([df_A, df_B], ignore_index=True)

In [ ]:
xaxis = df_AB.groupby("Atomic_number").mean().reset_index().to_numpy()[:, 0]
yaxis = df_AB.groupby("Atomic_number").mean().reset_index().to_numpy()[:, 1]

In the following plot, we see the average atomic density of ternary elements composed of Oxygen and another element, whose atomic number is given by the x-axis. Therefore, all values are averaged over the third element.

Plots show the predictions of the trained model on the test set, and on the kwnon values of the training set that are taken as reference values. Each point shows also the standard deviation. We emphasize that, considering that each value on the plot is given by an average over all elements in the periodic table, the standard deviation cannot go to zero by construction, even in the limit of taking all possible combinations. We then aim that averages and standard deviations predicted by our model are comparable to the ones of the reference model.


In [ ]:
plt.figure(figsize=(10, 6))

x = df_AB.groupby(["Atomic_number"]).mean().index.to_numpy().flatten()
y = df_AB.groupby(["Atomic_number"]).mean().to_numpy().flatten()
std = df_AB.groupby(["Atomic_number"]).std().to_numpy().flatten()

plt.errorbar(x, y, yerr=std.T, ls="", lw=3, marker="s", label="Reference value")
x = df_AB_pred.groupby(["Atomic_number"]).mean().index.to_numpy()
y = df_AB_pred.groupby(["Atomic_number"]).mean().to_numpy().flatten()
std = df_AB_pred.groupby(["Atomic_number"]).std().to_numpy().flatten()

plt.errorbar(x, y, yerr=std.T, ls="", lw=1.5, marker="s", label="AI prediction")

plt.legend(loc="upper right", fontsize="x-large")
plt.xlabel("Atomic number", fontsize="x-large")
plt.ylabel(r"Atomic density [$\AA^{-1}$]", fontsize="x-large")
plt.xticks([3, 11, 19, 37, 55, 89], fontsize="x-large")
plt.yticks(fontsize="x-large");

In the plot above, we can see that values predicted by the AI model are comparable to the reference values. In addition, we observe that the atomic density follows periodic trends, as values tend to be lower in the beginning and in the end of each row of the periodic table.
